In [38]:
# ===========================================
# PARAMÈTRES DE CONFIGURATION
# ===========================================

# Symbole à analyser
SYMBOLE = "AAPL"

# Date de fin d'analyse (format YYYY-MM-DD)
# Si None, utilise la date actuelle
DATE_FIN = None  # Exemple: "2024-12-31" ou None pour aujourd'hui

# Durée d'analyse (remonte dans le temps depuis DATE_FIN)
# Options: "1d", "5d", "1mo", "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", "max"
DUREE = "10d"

# Intervalle des données
# Options: "1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"
INTERVAL = "15m"

# Période de la moyenne mobile (en nombre de périodes)
MA_PERIODE = 30

print(f"Configuration:")
print(f"- Symbole: {SYMBOLE}")
print(f"- Date de fin: {DATE_FIN if DATE_FIN else 'Aujourd\'hui'}")
print(f"- Durée d'analyse: {DUREE}")
print(f"- Intervalle: {INTERVAL}")
print(f"- Moyenne mobile: {MA_PERIODE} périodes")
print()

# ===========================================
# RÉCUPÉRATION DES DONNÉES
# ===========================================

import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print(f"Récupération des données {SYMBOLE}...")

# Récupération des données selon les paramètres configurés
ticker = yf.Ticker(SYMBOLE)

if DATE_FIN:
    # Si une date de fin est spécifiée, calculer la date de début selon la durée
    date_fin = pd.to_datetime(DATE_FIN)
    
    # Conversion de la durée en jours approximatifs
    duree_map = {
        "1d": 1, "5d": 5, "1mo": 30, "3mo": 90, "6mo": 180,
        "1y": 365, "2y": 730, "5y": 1825, "10y": 3650
    }
    
    if DUREE in duree_map:
        jours = duree_map[DUREE]
        date_debut = date_fin - timedelta(days=jours)
        
        print(f"Téléchargement des données de {date_debut.strftime('%Y-%m-%d')} à {date_fin.strftime('%Y-%m-%d')}...")
        df_stock = ticker.history(start=date_debut.strftime('%Y-%m-%d'), 
                                  end=(date_fin + timedelta(days=1)).strftime('%Y-%m-%d'), 
                                  interval=INTERVAL)
    else:
        print(f"Téléchargement des données jusqu'à {date_fin.strftime('%Y-%m-%d')} ({DUREE})...")
        df_stock = ticker.history(period=DUREE, end=date_fin.strftime('%Y-%m-%d'), interval=INTERVAL)
else:
    # Utilisation de la période normale si pas de date de fin spécifiée
    print(f"Téléchargement des données historiques {SYMBOLE} ({DUREE})...")
    df_stock = ticker.history(period=DUREE, interval=INTERVAL)

# Vérification des données
if df_stock.empty:
    print(f"Erreur: Aucune donnée récupérée pour {SYMBOLE}")
else:
    print(f"\nDonnées {SYMBOLE} récupérées:")
    print(f"Période: {df_stock.index.min().strftime('%Y-%m-%d %H:%M')} à {df_stock.index.max().strftime('%Y-%m-%d %H:%M')}")
    print(f"Nombre de périodes ({INTERVAL}): {len(df_stock)}")
    print(f"Prix minimum: ${df_stock['Low'].min():.2f}")
    print(f"Prix maximum: ${df_stock['High'].max():.2f}")
    print(f"Prix de clôture le plus récent: ${df_stock['Close'].iloc[-1]:.2f}")
    
    # Affichage des premières et dernières lignes
    print(f"\nPremières données:")
    print(df_stock.head())
    print(f"\nDernières données (récentes):")
    print(df_stock.tail())
    
    # Informations sur les colonnes disponibles
    print(f"\nColonnes disponibles: {list(df_stock.columns)}")
    
    # Statistiques de base
    print(f"\nStatistiques de base du prix de clôture:")
    print(df_stock['Close'].describe())

Configuration:
- Symbole: AAPL
- Date de fin: Aujourd'hui
- Durée d'analyse: 10d
- Intervalle: 15m
- Moyenne mobile: 30 périodes

Récupération des données AAPL...
Téléchargement des données historiques AAPL (10d)...

Données AAPL récupérées:
Période: 2025-08-28 09:30 à 2025-09-11 15:30
Nombre de périodes (15m): 259
Prix minimum: $225.95
Prix maximum: $241.32
Prix de clôture le plus récent: $230.00

Premières données:
                                 Open        High         Low       Close  \
Datetime                                                                    
2025-08-28 09:30:00-04:00  230.809998  230.880005  229.335007  230.410507   
2025-08-28 09:45:00-04:00  230.440002  230.690002  229.679993  230.470001   
2025-08-28 10:00:00-04:00  230.429993  230.899994  230.020004  230.690002   
2025-08-28 10:15:00-04:00  230.705002  230.750000  230.279999  230.415802   
2025-08-28 10:30:00-04:00  230.430099  230.990005  230.350006  230.975006   

                            Volume  Div

In [39]:
# Graphique prix avec moyenne mobile seulement
import plotly.graph_objects as go
import pandas as pd

# Calcul de la moyenne mobile selon le paramètre configuré
df_stock[f'ma_{MA_PERIODE}'] = df_stock['Close'].rolling(window=MA_PERIODE).mean()

# Décalage de la moyenne mobile de PERIODE/2 vers la gauche
decalage = MA_PERIODE // 2
df_stock[f'ma_{MA_PERIODE}_shifted'] = df_stock[f'ma_{MA_PERIODE}'].shift(-decalage)

# Création du graphique principal avec seulement le prix et la moyenne mobile
fig_stock = go.Figure()

# Ajout du prix de clôture en ligne
fig_stock.add_trace(
    go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name=SYMBOLE,
        line=dict(color='#1f77b4', width=1)
    )
)

# Ajout de la moyenne mobile décalée
fig_stock.add_trace(
    go.Scatter(
        x=df_stock.index,
        y=df_stock[f'ma_{MA_PERIODE}_shifted'],
        mode='lines',
        name=f'MM {MA_PERIODE} périodes (décalée -{decalage})',
        line=dict(color='red', width=1),
        opacity=0.9
    )
)

# Configuration du layout
periode_text = f"jusqu'à {DATE_FIN}" if DATE_FIN else "à aujourd'hui"
fig_stock.update_layout(
    title={
        'text': f"{SYMBOLE} - Prix et Moyenne Mobile {MA_PERIODE} périodes décalée ({DUREE} {periode_text})",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    height=600,
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    ),
    xaxis_rangeslider_visible=False,
    template="plotly_white",
    margin=dict(l=50, r=50, t=80, b=50),
    yaxis_title="Prix (USD)",
    xaxis_title="Date"
)

# Affichage
config = {
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
}

print(f"Affichage du graphique {SYMBOLE} avec moyenne mobile {MA_PERIODE} périodes décalée de {decalage} périodes vers la gauche...")
print(f"Période analysée: {DUREE} {periode_text}")
fig_stock.show(config=config)

Affichage du graphique AAPL avec moyenne mobile 30 périodes décalée de 15 périodes vers la gauche...
Période analysée: 10d à aujourd'hui
